In [1]:
import torch
import torch.nn as nn
from torchvision import transforms 
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2 
import os


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# data_path = 'classify'
# for folder in os.listdir(data_path):
#     if folder == 'positive':
#         for file in os.listdir(os.path.join(data_path, folder)):
#             new_name = data_path + '\\1_' + file
#             curr_name = os.path.join(data_path, folder) + '\\' + file
#             os.renames(curr_name, new_name)
#     else:
#         for file in os.listdir(os.path.join(data_path, folder)):
#             new_name = data_path + '\\0_' + file
#             curr_name = os.path.join(data_path, folder) + '\\' + file
#             os.renames(curr_name, new_name)


In [4]:
class CustomDataset(Dataset):
    def __init__(self, path, transforms=None):
        self.names = os.listdir(path)
        self.transforms = transforms
        self.path = path

    def __len__(self):
        return len(self.names)

    def __getitem__(self, index):
        img_path = os.path.join(self.path, self.names[index])
        img = cv2.imread(img_path)
        label = int(self.names[index][0])
        if self.transforms:
            img = self.transforms(img) 
            
        return img, label

In [5]:
class ToTensor(object):
    def __call__(self, img):
        img = np.transpose(img, (2,0,1)).astype('float32')
        img = torch

        return img

class Resize(object):
    def __init__(self, size):
        assert isinstance(size, (int, tuple))
        self.size = size

    def __call__(self, img):

        h, w = img.shape[:2]
        if isinstance(self.size, int):
            if h > w:
                new_h, new_w = int(self.size * h / w), self.size
            else:
                new_h, new_w = self.size, int(self.size * w / h)
        else:
            new_h, new_w = self.size

        img = cv2.resize(img, (new_w,new_h))

        return img



In [6]:
transform = transforms.Compose([Resize((200, 512)), ToTensor()])

In [7]:
dataset = CustomDataset(path='classify', transforms=transform)
data_loader = DataLoader(dataset=dataset, batch_size=1, shuffle=True)

In [ ]:
for image, labels

In [13]:
class Model(nn.Module):
    def __init__(self, num_classes):
        super(Model, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, 5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, 5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2)
        )

        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, 5, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2)
        )

        self.flatten = nn.Flatten()
        self.fc = nn.Sequential(
            nn.Linear(102400, 32),
            nn.ReLU(),
            nn.Linear(32, num_classes),

        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.flatten(out)
        out = self.fc(out)
  
        return out
    


    

In [14]:
model = Model(2).to(device)
total_params = sum(p.numel() for p in model.parameters())

print(model)
print('Total params = ', total_params)

Model(
  (layer1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc): Sequential(
    (0): Linear(in_features=102400, out_features=32,

In [10]:
loss_func = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr = 0.00001)

In [11]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [17]:
total_step = len(data_loader)

for epoch in range(10):
    for images, labels in data_loader:
        images = images.to(device)
        labels = labels.to(device)

        #Forward pass
        output = model(images)
        loss = loss_func(output, labels)

        #Backward
        optim.zero_grad()
        loss.backward()
        optim.step()
    print('Epoch [{}/5], loss = {}'.format(epoch+1, loss))

Epoch [1/5], loss = 0.7929459810256958
